In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import csv
import warnings
import json
import threading
from selenium_recaptcha import Recaptcha_Solver
warnings.filterwarnings("ignore")

    Crawl data

    URL = https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html

Khởi tạo đối tượng webdriver và khai báo link URL

In [2]:
#URL = https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html
productLinks = []
nameFile = "dataLinkAloNhaDat.txt"
nameFileRS = "dataLinkAloNhaDatResult.txt"

Function resolve capcha

In [3]:
def resolveCapcha(browser): 
    try:
        btn = browser.find_element(By.CLASS_NAME, 'verify')
        solver = Recaptcha_Solver(
            driver= browser, # Your Web Driver
            debug=True,
        )
        solver.solve_recaptcha()
        btn.click()
    except:
        print("no capcha")
        pass

Lấy tất cả các link của bất động sản tại Đà Nẵng

In [ ]:
urlDaNang = "https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html/trang--{}.html"
file = open(nameFile, "a+")
browser = webdriver.Chrome(executable_path="./chromedriver.exe")
for pagenum in range(1, 1462):
   try:
      browser.get(urlDaNang.format(pagenum))
      if(len(browser.find_elements(By.CLASS_NAME, 'verify')) > 0):
         try:
            new_thred = threading.Thread(target=resolveCapcha, args=(browser,))
            new_thred.start()
            time.sleep(30)
         except:
            print("Error: unable to start thread")
            pass
      tagAs = browser.find_elements(By.LINK_TEXT, '<< xem chi tiết >>')
      for tagA in tagAs:
         link = tagA.get_attribute("href")
         print(link)
         file.write(link)
         file.write('\n')
         productLinks.append(link)
   except:
      continue
browser.quit()
print(len(productLinks))
file.close()
productLinks

Hàm đọc file to array

In [4]:
def readData(name):
    with open(name, 'r') as f:
        links = []
        for line in f.readlines():
            links.append(str(line))
    return links

In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(chrome_options=options, executable_path="chromedriver")
driver.get("https://sslproxies.org/")

ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 1]")))]
ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 2]")))]
driver.quit()
proxies = []
for i in range(0, len(ips)):
    proxies.append(ips[i]+':'+ports[i])
print(proxies)
for i in range(0, len(proxies)):
    try:
        print("Proxy selected: {}".format(proxies[i]))
        options = webdriver.ChromeOptions()
        options.add_argument('--proxy-server={}'.format(proxies[i]))
        driver = webdriver.Chrome(options=options, executable_path="chromedriver")
        driver.get("https://www.whatismyip.com/proxy-check/?iref=home")
        if "Proxy Type" in WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "p.card-text"))):
            break
    except Exception:
        driver.quit()
print("Proxy Invoked")

TimeoutException: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D4DCE3+50899]
	(No symbol) [0x00CDE111]
	(No symbol) [0x00BE5588]
	(No symbol) [0x00C108F9]
	(No symbol) [0x00C10AFB]
	(No symbol) [0x00C3F902]
	(No symbol) [0x00C2B944]
	(No symbol) [0x00C3E01C]
	(No symbol) [0x00C2B6F6]
	(No symbol) [0x00C07708]
	(No symbol) [0x00C0886D]
	GetHandleVerifier [0x00FB3EAE+2566302]
	GetHandleVerifier [0x00FE92B1+2784417]
	GetHandleVerifier [0x00FE327C+2759788]
	GetHandleVerifier [0x00DE5740+672048]
	(No symbol) [0x00CE8872]
	(No symbol) [0x00CE41C8]
	(No symbol) [0x00CE42AB]
	(No symbol) [0x00CD71B7]
	BaseThreadInitThunk [0x762F7D49+25]
	RtlInitializeExceptionChain [0x771AB74B+107]
	RtlClearBits [0x771AB6CF+191]


Lấy các thuộc tính của bất động sản

In [5]:
detailProductsAloDN = []
browser = webdriver.Chrome(executable_path="chromedriver")
links = readData(nameFile)
for link in links:
    try:
        browser.get(link)
        if(len(browser.find_elements(By.CLASS_NAME, 'verify')) > 0):
            try:
                new_thred = threading.Thread(target=resolveCapcha, args=(browser,))
                new_thred.start()
                time.sleep(30)
            except:
                print("Error: unable to start thread")
                pass
        data = browser.find_element(By.XPATH, "//div[@class='content plp']")
        try:
            name = data.find_element(By.TAG_NAME, "h1").text
            detailProduct = {"name": name}
        except:
            pass
        try:
            price = data.find_element(By.CLASS_NAME, "price").find_element(By.CLASS_NAME, "value").text
            detailProduct["Giá"] = price
        except:
            pass
        try:
            square = data.find_element(By.CLASS_NAME, "square").find_element(By.CLASS_NAME, "value").text
            detailProduct["Diện tích"] = square
        except:
            pass
        try:
            address = data.find_element(By.CLASS_NAME, "address").find_element(By.CLASS_NAME, "value").text
            detailProduct["Địa chỉ"] = address
        except:
            pass
        try:
            infor =  data.find_elements(By.TAG_NAME, "td")
            for i in range(0, len(infor), 2):
                detailProduct[infor[i].text] = infor[i + 1].text if(infor[i + 1].text != '') else 'true'
        except:
            pass
        fileRS = open(nameFileRS, "a+", encoding="utf-8")
        fileRS.write(json.dumps(detailProduct, ensure_ascii=False))
        fileRS.write('\n')
        fileRS.close()
        detailProductsAloDN.append(detailProduct)
        print(detailProduct)
    except Exception as e:
        print(e)
        continue
browser.quit()
detailProductsAloDN

{'name': 'Bán đất đường An Dương Vương,Mỹ An, Ngũ Hành Sơn, Đà Nẵng kế bên dự án đang bán Sun Group', 'Giá': '7 tỷ', 'Diện tích': '80 m\n2', 'Địa chỉ': 'Đường An Dương Vương, Phường Mỹ An, Quận Ngũ Hành Sơn, Đà Nẵng', 'Mã tin': '12709760', 'Hướng': '_', 'Phòng ăn': '---', 'Loại tin': 'Cần bán', 'Đường trước nhà': '30m', 'Nhà bếp': '---', 'Loại BDS': 'Đất thổ cư, đất ở', 'Pháp lý': '---', 'Sân thượng': '---', 'Chiều ngang': '5m', 'Số lầu': '---', 'Chổ để xe hơi': '---', 'Chiều dài': '---', 'Số phòng ngủ': '---', 'Chính chủ': '---'}
{'name': 'Cần bán lô đất 114m2 Kiệt Nguyễn Văn Thoại P. Mỹ An, Q. Ngũ Hành Sơn', 'Giá': '4,4 tỷ', 'Diện tích': '114 m\n2', 'Địa chỉ': 'Đường Nguyễn Văn Thoại, Phường Mỹ An, Quận Ngũ Hành Sơn, Đà Nẵng', 'Mã tin': '12709837', 'Hướng': '_', 'Phòng ăn': '---', 'Loại tin': 'Cần bán', 'Đường trước nhà': '3,5m', 'Nhà bếp': '---', 'Loại BDS': 'Đất thổ cư, đất ở', 'Pháp lý': '---', 'Sân thượng': '---', 'Chiều ngang': '5m', 'Số lầu': '---', 'Chổ để xe hơi': '---', 'Chi

KeyboardInterrupt: 

Ghi dữ liệu qua file excel

In [ ]:
csv_columns = []
for product in detailProductsAloDN:
    tempkhac = set(list(product)[0:]) ^ set(csv_columns)
    tempgiong = set(list(product)[0:]) & set(csv_columns)
    csv_columns = tempkhac ^ tempgiong
csv_file = "dataAloNhaDat.csv"
try:
    with open(csv_file, 'w+',encoding= "utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for dataz in detailProductsAloDN:
            try:
                writer.writerow(dataz)
            except:
                continue
except IOError:
    print("I/O error")